In [ ]:
%%capture
!pip install jsonlines
!pip install evaluate
!pip install rouge_score
!pip install bert_score
!pip install unsloth

In [ ]:
!unzip /content/results.zip

Archive:  /content/results.zip
 extracting: huggingface_tokenizers_cache/.locks/models--unsloth--meta-llama-3.1-8b-instruct-bnb-4bit/165b36bc2293dda9a2fb3c0daf6577d9eba9df7a.lock  
 extracting: huggingface_tokenizers_cache/.locks/models--unsloth--meta-llama-3.1-8b-instruct-bnb-4bit/5cc5f00a5b203e90a27a3bd60d1ec393b07971e8.lock  
 extracting: huggingface_tokenizers_cache/.locks/models--unsloth--meta-llama-3.1-8b-instruct-bnb-4bit/d185c9e9b9d1e3f4e0c613d27efc412efdc3dde1.lock  
 extracting: huggingface_tokenizers_cache/models--unsloth--meta-llama-3.1-8b-instruct-bnb-4bit/.no_exist/5b0dd3039c312969e7950951486714bff26f0822/added_tokens.json  
  inflating: huggingface_tokenizers_cache/models--unsloth--meta-llama-3.1-8b-instruct-bnb-4bit/blobs/165b36bc2293dda9a2fb3c0daf6577d9eba9df7a  
  inflating: huggingface_tokenizers_cache/models--unsloth--meta-llama-3.1-8b-instruct-bnb-4bit/blobs/5cc5f00a5b203e90a27a3bd60d1ec393b07971e8  
  inflating: huggingface_tokenizers_cache/models--unsloth--meta-l

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.2: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers via:
`pip uninstall transformers -y && pip install --upgrade --no-cache-dir "git+https://github.com/huggingface/transformers.git"`
Unsloth 2024.10.2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.


In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp`

In [ ]:
print(tokenizer._ollama_modelfile)